In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt
import riskfolio.Portfolio as pf


import empyrical.stats as em
import quantstats as qs
import pyfolio as pyf
%matplotlib inline

import time

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State


#Added by Ji Wu
plt.style.use('fivethirtyeight')
np.random.seed(777)


import warnings
warnings.filterwarnings("ignore")

yf.pdr_override()
# pd.options.display.float_format = '{:.4%}'.format


C:\Users\titas\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
C:\Users\titas\Anaconda3\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
# Date range
start =  '2000-01-03'  #POS'2000-01-03' #2001-01-11 
end =   '2017-12-31' #POS'2017-01-31' #'2018-01-02' 


# Tickers of stocks - Inputted from dashboard
stocks = ['BAC', 'C', 'SNV', 'STI', 'WFC', 'LNC', 'PGR', 'GL', 'GS',
                  'SCHW', 'AXP', 'BEN', 'BLK', 'COF', 'MS']
stocks.sort()


#With SPY
assets = stocks.copy()
assets.append('SPY')

   
# Downloading data
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Adj Close', slice(None))]
data.columns = assets


# Calculating returns
Return_Data = data.pct_change().dropna()
# Return_Data.head()

[*********************100%***********************]  16 of 16 completed


In [3]:
window = 1500
num_portfolios = 2000
month = 22
# Return_Data[ticker_list]
# ticker_list

In [20]:
#Function to calculate COSR weights
def CoSR_weights(i,c, tickers):
    index = i
    port_CoSR_ratio = []
    port_volatility = []
    stock_weights = []
    weights = []
    stocks = tickers

#     df = Return_Data[i:i+window]
    
    df_return_C = pd.DataFrame(columns = assets)
    df = Return_Data[i:i+window].iloc[4:]
    
    a = 0
    while a < len(df):
        if qs.stats.comp(df[a:a+month]['SPY']) < c:
            df_return_C = df_return_C.append(df[a:a+month])
        a= a+month

    #Filtering based on ticker_list
    ticker_spy = tickers.copy()
    ticker_spy.append('SPY')
    df  = df_return_C[ticker_spy]
    Y_SPY = pd.DataFrame(df['SPY'], columns = ['SPY'])
    Y = df[stocks] 
    
    
    cov_matrix = Y.cov()
    mean_returns = Y.mean()
    mkt_returns = Y_SPY.SPY.mean()

    for i in range(num_portfolios):
        weights = np.random.random(len(stocks))
        weights /= np.sum(weights)
        mkt_returns = df['SPY'].mean()
        returns = np.dot(weights, mean_returns)
        volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        CoSR_ratio = (returns - mkt_returns)/volatility
        port_CoSR_ratio.append(CoSR_ratio)
        stock_weights.append(weights)
        
    weight = stock_weights[port_CoSR_ratio.index(max(port_CoSR_ratio))]

    df = pd.DataFrame(data = weight, index = stocks).T
  
    #print(index+window+month + 1 )
    #print(len(data)-(index+window+1))
    if index+window+month + 1 < len(data):
        newdf = pd.DataFrame(np.repeat(df.values,month,axis=0))
        newdf.columns = df.columns
        newdf['Date'] = data.index.values[index+window+1:index+window+month + 1]
    else:
        newdf = pd.DataFrame(np.repeat(df.values,len(data)-(index+window+1),axis=0))
        newdf.columns = df.columns
        newdf['Date'] = data.index.values[index+window+1:len(data)]
    
    newdf= newdf.set_index('Date')
    return newdf

In [5]:
#Function to calculate SR weights
def SR(index, tickers):

    #Slicing the data
    Y = Return_Data[index:index+window].drop(columns=['SPY'])

    #Filtering based on ticker_list
    Y = Y[tickers]
     
    # Building the portfolio object
    port = pf.Portfolio(returns=Y)
    # Calculating optimum portfolio

    # Select method and estimate input parameters:

    method_mu='hist' # Method to estimate expected returns based on historical data.
    method_cov='hist' # Method to estimate covariance matrix based on historical data.

    port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

    # Configuring short weights options

#     port.sht = True # Allows to use Short Weights
#     port.uppersht = 0.3 # Maximum value of sum of short weights in absolute value

    # Estimate optimal portfolio:

    model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
    rm = 'MV' # Risk measure used, this time will be variance
    obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
    hist = True # Use historical scenarios for risk measures that depend on scenarios
    rf = 0 # Risk free rate
    l = 0 # Risk aversion factor, only useful when obj is 'Utility'

    w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    
    df = w.T
     
 
    if index+window+month + 1 < len(data):
        newdf = pd.DataFrame(np.repeat(df.values,month,axis=0))
        newdf.columns = df.columns
        newdf['Date'] = data.index.values[index+window+1:index+window+month + 1]
    else:
        newdf = pd.DataFrame(np.repeat(df.values,len(data)-(index+window+1),axis=0))
        newdf.columns = df.columns
        newdf['Date'] = data.index.values[index+window+1:len(data)]
    
    newdf= newdf.set_index('Date')
        
    return newdf

In [6]:
#Function to calculate weights for each portfolio
def weights(tickers):
    #Computing weights based on COSR/ SR/ 1/n
    df_CoSRweights = pd.DataFrame(columns = tickers)
    df_SR_weights = pd.DataFrame(columns = tickers)

    #Set the value of C
#     c = -0.067
    c = 0.0

    i = 0
    while i <= len(Return_Data.index.values)-window:
#         print (i)
        weight_CoSR = CoSR_weights(i,c, tickers) 
        df_CoSRweights= df_CoSRweights.append(weight_CoSR)
#         tickers = tickers.drop(columns=['SPY'])
        weight_SR = SR(i,tickers)
        df_SR_weights= df_SR_weights.append(weight_SR)
        i = i + month
    return df_CoSRweights, df_SR_weights

In [7]:
def return_metrics (tickers):
    #Individual returns over the period
    i_returns = Return_Data[1500:len(data)].drop(columns=['SPY'])
    i_returns = i_returns[tickers]

    #Calling the weights function

    df_CoSRweights, df_SR_weights = weights(tickers)

    #Computing SR & equally weighted portfolio returns portfolio returns
    p_cosr_returns = df_CoSRweights[tickers[0]]*i_returns[tickers[0]]
    p_sr_returns = df_SR_weights[tickers[0]]*i_returns[tickers[0]]
    p_ew_returns = (1/len(tickers))*i_returns[tickers[0]]
    i=1
    while i < len(tickers):
        p_cosr_returns = p_cosr_returns + df_CoSRweights[tickers[i]]*i_returns[tickers[i]]
        p_sr_returns = p_sr_returns + df_SR_weights[tickers[i]]*i_returns[tickers[i]]
        p_ew_returns = p_ew_returns + (1/len(tickers))*i_returns[tickers[i]]
        i = i + 1
        
    #Return-Cumulative returns Dataframe    
    df_return = pd.DataFrame({'COSR_Returns': p_cosr_returns, 'SR_Returns': p_sr_returns,
                   'EW_Returns': p_ew_returns, 'Cum_COSR_Returns': p_cosr_returns.add(1).cumprod(),
                   'Cum_SR_Returns': p_sr_returns.add(1).cumprod(),'Cum_EW_Returns': p_ew_returns.add(1).cumprod(),
                  })     
        
    #Calculatig metrics    
    df_metrics = pd.DataFrame(columns=['metric','p_cosr_returns', 'p_sr_returns', 'p_ew_returns'])
    metrics= [f for f in dir(qs.stats) if f[0] != '_']
    for i in range(0,len(metrics)):
        if metrics[i] not in ['compare', 'consecutive_losses', 'consecutive_wins', 'greeks', 'information_ratio', 
                              'pct_rank', 'r2', 'r_squared', 'rolling_greeks']:
            metric = metrics[i]
            cosr = eval('qs.stats.'+metrics[i]+'(p_cosr_returns)')
            sr = eval('qs.stats.'+metrics[i]+'(p_sr_returns)')
            ew = eval('qs.stats.'+metrics[i]+'(p_ew_returns)')
            df_metrics = df_metrics.append({'metric':metric, 'p_cosr_returns':cosr, 'p_sr_returns':sr, 'p_ew_returns':ew}, ignore_index=True)

    df_metrics= df_metrics.set_index('metric')
    
    
    return df_CoSRweights, df_SR_weights, df_return, df_metrics 

In [19]:
#Dash dropdown menu
options = []
for tic in stocks:
    #{'label': 'user sees', 'value': 'script sees'}
    mydict = {}
    mydict['label'] = tic #Apple Co. AAPL
    mydict['value'] = tic
    options.append(mydict)

In [16]:
app = dash.Dash()

In [17]:
app.layout = html.Div([
    html.H1('Portfolio Dashboard', style={'textAlign': 'center'}),
    dcc.Markdown(''' --- '''), 
    html.H1('Systemic Risk Portfolio Analysis'),
    html.H3('Enter a stock symbol:', style={'paddingRight': '30px'}),
    dcc.Dropdown(id='input-1-state',options = options, multi = True ),
    html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
    dcc.Graph(id='cum_return'),
    dcc.Graph(id='return_metrics'),
    dcc.Graph(id='cosr'),
    dcc.Graph(id='sr')
])


@app.callback([ Output('cum_return', 'figure'), Output('return_metrics', 'figure'), 
               Output('cosr', 'figure'), Output('sr', 'figure')], 
              [Input('submit-button-state', 'n_clicks')],
              [State('input-1-state', 'value')])
def update_output(n_clicks, input1):

    df_CoSRweights, df_SR_weights, df_return, df_metrics  = return_metrics(input1)

#     Cumulative Returns
    fig_cum = px.line(df_return, y = ['Cum_COSR_Returns', 'Cum_SR_Returns', 'Cum_EW_Returns'])

    fig_cum.update_layout(
    title="Cumulative returns from "+str(start)+" - "+str(end),
    xaxis_title="Time",
    yaxis_title="Weights",
    legend_title="Weighting method")
    

    
    #Metrics
    fig = go.Figure(data=[go.Table(
    header=dict(values=["Metrics","COSR Portfolio", "SR Portfolio", "Equally-Weighted portfolio"],
               fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_metrics.index.values, df_metrics.p_cosr_returns, df_metrics.p_sr_returns, df_metrics.p_ew_returns],
               fill_color='lavender',
               align='left'))
               ])
     
    
    #CoSR Weights
    fig_cosr = px.line(df_CoSRweights, y = df_CoSRweights.columns)
    fig_cosr.update_layout(
    title="COSR Weights from "+str(start)+"-"+str(end),
    xaxis_title="Time",
    yaxis_title="Weights",
    legend_title="Tickers",
    )

    #SR Weights
    fig_sr = px.line(df_SR_weights, y = df_SR_weights.columns)
    fig_sr.update_layout(
    title="SR Weights from "+str(start)+"-"+str(end),
    xaxis_title="Time",
    yaxis_title="Weights",
    legend_title="Tickers",
    )
    
    return fig_cum, fig, fig_cosr, fig_sr

In [18]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Aug/2020 21:16:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2020 21:16:19] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Aug/2020 21:16:19] "GET /_dash-dependencies HTTP/1.1" 200 -
[2020-08-29 21:16:19,740] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\titas\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: None

During handling of the above exception, another exception